# Twitter

## Kelly Shaffer, Raphael Nash, Xiaomeng (Vivian) Kong, Michael Muller, Blandon Casenave 

In [2]:
import networkx as nx
import pandas as pd
import urllib
import numpy as np
import matplotlib.pyplot as plt 
import sys
import os
from networkx.algorithms import bipartite as bi
#twitter OAUTH 
import time as tm
import json
import argparse as ag
import tweepy as tw
from tweepy.streaming import StreamListener as sl
from tweepy import OAuthHandler as oa
from tweepy import Stream as st
import oauth2 as oauth
import re
import dateutil.parser as dp
from pytz import timezone as tz
import pytz 

In [5]:
CONSUMER_KEY = "vVgMvZu4a5UFuNNmIqLG6kdst"
CONSUMER_SECRET = "SjEZQJIcMjSyP6Tj2blyJsrqfmtsHZmIaJldT0hlbpW4NNx2a3"
ACCESS_KEY = "199763323-wPFrK2QTS9HiMyOnY2yfS1CCCQEGBaS0ZTG0YpfJ"
ACCESS_SECRET = "CPowEv0tAn5Hka5ayCjlGcYl2kecvTJwqiQuSa0OYuF4m"

consumer = oauth.Consumer(key=CONSUMER_KEY, secret=CONSUMER_SECRET)
access_token = oauth.Token(key=ACCESS_KEY, secret=ACCESS_SECRET)
client = oauth.Client(consumer, access_token)

In [6]:
OAUTH_KEYS = {'consumer_key':CONSUMER_KEY, 'consumer_secret':CONSUMER_SECRET,
 'access_token_key':ACCESS_KEY, 'access_token_secret':ACCESS_SECRET}
auth = tw.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [152]:
search = tw.Cursor(api.search, q='#donaldtrump').items(400)

In [153]:
sn = []
text = []
timestamp =[]
for tweet in search:
    print tweet.user.screen_name, tweet.created_at, tweet.text
    timestamp.append(tweet.created_at)
    sn.append(tweet.user.screen_name)
    text.append(tweet.text)

Financeolizer 2017-07-17 00:46:31 Trump lashes out as new poll finds record low approval rating #President #DonaldTrump https://t.co/VUxIgwCvI3 https://t.co/rCBt3O9HL7
MrScavellz 2017-07-17 00:46:30 Trump still hasn't acknowledged McCain having surgery but he has found plenty of time to tweet about himself &amp; insult the media #DonaldTrump
dp42049 2017-07-17 00:46:19 RT @ShirlsAdams: @20thgrader @20committee @lulu_lemew My gawd, #DonaldTrump was talking about Russia when he wrote this tweet in 2013!
KingOfTerror201 2017-07-17 00:46:10 Younus AlGohar's Solution to #Terrorism https://t.co/lE1dwkin0E #Daesh #ISIS ##BokoHaram #Taliban #AlNusra #AlQaeda #DonaldTrump
SubiksaS 2017-07-17 00:46:00 RT @Timesolizer: #Russia, others intervened in US #presidentialelection: #DonaldTrump - #TimesofIndia... https://t.co/T2yeifV36J https://t.…
roldandin 2017-07-17 00:45:29 Reebok schools us (I'm belong polite).  #donaldtrump #covfefe #macron #reebok #youreinsuchgoodshape https://t.co/8QrfCXHBzQ
arti

Tea_Party_Chris 2017-07-17 00:33:14 RT @nancy_us85: One Small North #dakota Town Taught Mark Zuckerberg Exactly Why #DonaldTrump Is #President
https://t.co/gk0Y7pSr7R
#SundayF…
Syedraza8191 2017-07-17 00:33:12 The Reality of #Wahhabism in #Islam https://t.co/G6VMOs5mqS #SaudiArabia #Salafism #Deobandi #ISIS #Taliban #AlQaeda #DonaldTrump #MikePence
trumpnews247 2017-07-17 00:33:01 #DonaldTrump -  Trump's travel ban to head back to Supreme Court - Washington Times https://t.co/kRsFZL4on9   - - - #trump
WomensPowerCen 2017-07-17 00:32:02 #americanelection #RepublicansForHillary Empowering site:Can #DonaldTrump win #USElections https://t.co/mg0jk9kAWD https://t.co/9JMGxRHG1L
FireTrumpPence 2017-07-17 00:31:35 @JoyAnnReid #DonaldTrump #CowardInChief #LockHimUp https://t.co/rqHXw7ijbn
mommala728 2017-07-17 00:31:30 RT @amjoyshow: Thoughts? → #DonaldTrump begs Theresa May to fix a 'warm welcome' for his state visit https://t.co/fZMVmIt1FT
R1Ray 2017-07-17 00:31:26 RT @JamesReader_RP: #Donald

SumairSg 2017-07-17 00:05:34 RT @rafu007: The Mischief of the Pseudo #Muslims https://t.co/aHrad9F7Hd #ISIS #Taliban #BokoHaram #AlShabab #AlQaeda #AlNusra #DonaldTrump…
shaheen_KCH 2017-07-17 00:05:33 RT @rafu007: The Mischief of the Pseudo #Muslims https://t.co/aHrad9F7Hd #ISIS #Taliban #BokoHaram #AlShabab #AlQaeda #AlNusra #DonaldTrump…
Hamad_Qasir 2017-07-17 00:05:32 RT @rafu007: The Mischief of the Pseudo #Muslims https://t.co/aHrad9F7Hd #ISIS #Taliban #BokoHaram #AlShabab #AlQaeda #AlNusra #DonaldTrump…
LIONGSY 2017-07-17 00:05:32 RT @rafu007: The Mischief of the Pseudo #Muslims https://t.co/aHrad9F7Hd #ISIS #Taliban #BokoHaram #AlShabab #AlQaeda #AlNusra #DonaldTrump…
Hashir_Khan2 2017-07-17 00:05:31 RT @rafu007: The Mischief of the Pseudo #Muslims https://t.co/aHrad9F7Hd #ISIS #Taliban #BokoHaram #AlShabab #AlQaeda #AlNusra #DonaldTrump…
Aahil_IbrahimK 2017-07-17 00:05:31 RT @rafu007: The Mischief of the Pseudo #Muslims https://t.co/aHrad9F7Hd #ISIS #Taliban #BokoHaram #AlShab

eqtr8er 2017-07-16 23:58:14 RT @JamesReader_RP: #DonaldTrump’s Sunday Tweet About His Approval Rating is Complete, Total, Delusional Horseshit &gt;&gt;https://t.co/EPYVjn34Z…
GeeLauder 2017-07-16 23:57:38 RT @amjoyshow: Thoughts? → #DonaldTrump begs Theresa May to fix a 'warm welcome' for his state visit https://t.co/fZMVmIt1FT
ozair_kol 2017-07-16 23:57:01 An Introduction to #Wahhabism https://t.co/2KmUepZbm6 #SaudiArabia #Salafism #Deobandi #ISIS #Taliban #BokoHaram #DonaldTrump
1rizwan_shoukat 2017-07-16 23:56:19 RT @rafu007: The Mischief of the Pseudo #Muslims https://t.co/XJJZ5DyZ65 #ISIS #Taliban #BokoHaram #AlShabab #AlQaeda #AlNusra #DonaldTrump
FaisalBashir_pk 2017-07-16 23:56:19 RT @rafu007: The Mischief of the Pseudo #Muslims https://t.co/XJJZ5DyZ65 #ISIS #Taliban #BokoHaram #AlShabab #AlQaeda #AlNusra #DonaldTrump
Rehman_ali0 2017-07-16 23:56:18 RT @rafu007: The Mischief of the Pseudo #Muslims https://t.co/XJJZ5DyZ65 #ISIS #Taliban #BokoHaram #AlShabab #AlQaeda #AlNusra #Do

MrCyberdude 2017-07-16 23:42:14 RT @chrisroytaylor: Great minds think alike!!!

#Armageddon #missiles #KimJongUn #NorthKorea #ICBM
#Trump #DonaldTrump @theheraldsun @carto…
JuliaLeo4 2017-07-16 23:41:51 RT @amjoyshow: .@TIMOBRIEN shares his views on Felix Sater, his reported connections to #DonaldTrump &amp; why you want to know #AMJoy https://…
gyaleemgohar 2017-07-16 23:40:57 RT @rafu007: An Introduction to #Wahhabism https://t.co/Yn7Y6Yevig #SaudiArabia #Salafism #Deobandi #ISIS #Taliban #BokoHaram #DonaldTrump…
matuoka7ocean 2017-07-16 23:40:43 THE DAILY   Donald Trump, #DonaldTrump #DonaldTrumpsFamily #DonaldTrumpWomenSupporter 紙が更新されました！ https://t.co/OhEZ0mXgmm #trump
KnowVidLeaks 2017-07-16 23:40:32 President Trump Upstaged By Easter Bunny (pls RT↺❤️) ️️ https://t.co/7mGdx8vzia #DonaldTrump
Vmbritsch 2017-07-16 23:40:16 RT @amjoyshow: .@TIMOBRIEN shares his views on Felix Sater, his reported connections to #DonaldTrump &amp; why you want to know #AMJoy https://…
JcKatherine 2017

ricklevy67 2017-07-16 23:17:16 #DonaldTrump: Equally Hates #Jews And #Muslims @nigusanegast #TheResistance @randlight @nobby15 @RBJRON https://t.co/n9N9Fn87jM
grgesq 2017-07-16 23:16:23 RT @glamelegance: @CNNPolitics Here's a theme for #DonaldTrump ... Tell The Truth Week! 🤡 #TrumpCrimeFamily #TrumpLies
LengelDavid 2017-07-16 23:16:20 He's wearing a #swastika t in the #NYC #Brooklyn #subway, speaking about being sick of hearing about #DonaldTrump a… https://t.co/is3tq0nYaP
GhulamRiaz47 2017-07-16 23:15:51 RT @AntiTalibanISIS: What is #Wahhabism? https://t.co/PGJYm8VQi3 #Salafism #Deobandi #ISIS #Taliban #Islam #Quran #DonaldTrump #MikePence #…
TrishDickerson 2017-07-16 23:15:50 Why are we not more appalled? Why is there even a hint that some think it's okay. #DonaldTrump #RussiaGate #treason #TrumpRussia
GJapproved 2017-07-16 23:15:41 Trumps attorneys throwing secret service under the bus as a defense to the collusion debacle. Smh.. dude has zero spine #donaldtrump
BeachyLifey 2017-07-

No2TrumpBot 2017-07-16 23:00:02 RT @sarusk: Donald Trump is not a #Christian -Trump worships Trump. #JimmyCarter #DonaldTrump #Christian #TrumpCrimeFamily #TrumpLies #25th…
KatleXW 2017-07-16 22:59:43 RT @NotifyrInc: #DonaldTrump approval rating at 70-year low as Russia scandal swirls  #AmericanPolitics https://t.co/5FZSqeVBU5
sarusk 2017-07-16 22:59:40 Donald Trump is not a #Christian -Trump worships Trump. #JimmyCarter #DonaldTrump #Christian #TrumpCrimeFamily… https://t.co/GkkMSYrCnL
KorenZailckas 2017-07-16 22:58:34 Child #1 asked this question. Before Siri could answer, child #2 said #DonaldTrump https://t.co/RuG39yB7bK
lovetybeega 2017-07-16 22:58:26 RT @glamelegance: @Acosta #DonaldTrump ostracized the Intel Community and now trying to do it to USSS. Seems no one is off limits. Sad.
Ernestsworld 2017-07-16 22:57:56 #RudyGiuliani has never been this 
quiet on anything #DonaldTrump. 
He is up to his neck in this collusion. 
@nytimes @washingtonpost #amjoy
SydneeS_43 2017-07-16 22:

In [154]:

# Convert lists to dataframe
df = pd.DataFrame()
df['timestamp'] = timestamp
df['sn'] = sn
df['text'] = text

In [155]:
# Prepare ford date filtering. Adding an EST time column since chat hosted by people in that time zone.
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['EST'] = df['timestamp'] - pd.Timedelta(hours=5) #Convert to EST

In [156]:
df['EST'] = pd.to_datetime(df['EST'])

In [157]:
#Subset for the dates required. Can select a specific date or time to examine.
#import time
#df = df[(pd.to_datetime("2017-7-14 20:00:00", format='%Y-%m-%d %H:%M:%S') < df['EST']) & (df['EST'] < pd.to_datetime("2017-7-14 21:00:00", format='%Y-%m-%d %H:%M:%S'))]

In [177]:
# adjust local path.
df.to_csv('/Users/digitalmarketer1977/Desktop/620trumptweets.csv',index = False,encoding='utf-8')

In [159]:
# Create a list of the unique usernames in order to see which users we need to retrieve friends for.
allNames = list(df['sn'].unique())

In [162]:
# Initialize dataframe of users that will hold the edge relationships
dfUsers = pd.DataFrame()
dfUsers['userFromName'] =[]
dfUsers['userFromId'] =[]
dfUsers['userToId'] = []
count = 0 

In [167]:
nameCount = len(allNames)
# The choice to retrieve friends (who the user is following) rather than followers is intentional.
# Either would work. However, many Twitter users follow fewer users than are following them, especially the most popular accounts. 
# This reduces the number of very large calls to Twitter API, which seemed to cause problems.
for name in allNames:
    # Build list of friends    
    currentFriends = []
    for page in tw.Cursor(api.friends_ids, screen_name=name).pages():
        currentFriends.extend(page)
    currentId = api.get_user(screen_name=name).id
    currentId = [currentId] * len(currentFriends)
    currentName = [name] * len(currentFriends)   
    dfTemp = pd.DataFrame()
    dfTemp['userFromName'] = currentName
    dfTemp['userFromId'] = currentId
    dfTemp['userToId'] = currentFriends
    dfUsers = pd.concat([dfUsers,dfTemp])
    time.sleep(5) # avoids hitting Twitter rate limit
    # Progress bar to track approximate progress
    count +=1
    per = round(count*100.0/nameCount,1)
    sys.stdout.write("\rTwitter call %s%% complete." % per)
    sys.stdout.flush() 

Rate limit reached. Sleeping for: 95
Twitter call 4.7% complete.Rate limit reached. Sleeping for: 826
Twitter call 9.7% complete.Rate limit reached. Sleeping for: 819
Twitter call 14.8% complete.Rate limit reached. Sleeping for: 821


KeyboardInterrupt: 

In [168]:
# Again, to limit the number of calls to Twitter API, just do lookups on followers that connect to those in our user group.
# We are not interested in "friends" that are not part of this community.
fromId = dfUsers['userFromId'].unique()
dfChat = dfUsers[dfUsers['userToId'].apply(lambda x: x in fromId)]

In [169]:
# No more Twitter API lookups are necessary. Create a lookup table that we will use to get the verify the userToName
dfLookup = dfChat[['userFromName','userFromId']]
dfLookup = dfLookup.drop_duplicates()
dfLookup.columns = ['userToName','userToId']
#dfCommunity = pd.DataFrame()
#dfCommunity = pd.merge(dfLookup, on='userToId')

In [178]:
dfLookup

,userToName,userToId
9,KingOfTerror201,2.825304e+09
7322,artistproco,4.790560e+08
187,socialcompleta,3.897022e+09
8,Kashif_Noor_MHD,2.826542e+09
5,1rizwan_shoukat,2.785601e+09
10,FaisalBashir_pk,2.812415e+09
4,Rehman_ali0,2.568125e+09
6,Haashir_Ali,2.809162e+09
10,Yasirali093,2.795820e+09
5,Sameer_Shakir1,2.797295e+09


In [171]:
dfLookup.to_csv('dfLookup.csv',index = False,encoding='utf-8')

In [175]:
dfGraph = pd.read_csv('dfLookup.csv')

In [176]:
#dfGraph = dfGraph[index]

NameError: name 'index' is not defined

In [1]:
G = nx.DiGraph()
#G = nx.Graph()
G.add_nodes_from(df['userFromId'])
#G.add_edges_from(zip(df['userFromId'],df['userToId']))

NameError: name 'nx' is not defined

In [ ]:
temp = zip(df['userFromId'],df['userToId'])
G.add_edges_from(temp)

In [ ]:
# Give nodes their Usernames
dfLookup = df[['userFromName','userFromId']].drop_duplicates()

In [ ]:
dfLookup.head()
for userId in dfLookup['userFromId']:
    temp = dfLookup['userFromName'][df['userFromId']==userId]
    G.node[userId]['userName'] = temp.values[0]

In [ ]:
nx.draw(G, pos=nx.spring_layout(G,k=.12),node_color='c',edge_color='k')

## Code Sources

[https://stackoverflow.com/questions/6399978/getting-started-with-twitter-oauth2-python](Getting started with Twitter\OAuth2\Python)

[http://www.techpoweredmath.com/constructing-social-graph-twitter-plotly/#.WWuUx9MrKYW](Constructing a Social Graph With Twitter and Plotly)